In [69]:
import pandas as pd
import numpy as np
from arch import arch_model
from statsmodels import regression
from scipy import linalg
import re

In [70]:
data = pd.read_csv("oil_futures.csv")
log_r = data.log_return
log_r = log_r[log_r!=0]

In [71]:
garch11 = arch_model(log_r*1000, p=1,q=1)
res11 = garch11.fit(update_freq=5)

Iteration:      5,   Func. Count:     31,   Neg. LLF: 5579.6220064688105
Iteration:     10,   Func. Count:     61,   Neg. LLF: 5578.24538662792
Iteration:     15,   Func. Count:    107,   Neg. LLF: 5248.694186650964
Iteration:     20,   Func. Count:    141,   Neg. LLF: 5136.633117818144
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 5133.7662165158945
            Iterations: 24
            Function evaluations: 164
            Gradient evaluations: 23


In [72]:
res.params

mu         -8.465481e-01
omega       4.441352e+00
alpha[1]    6.321265e-09
beta[1]     1.000000e+00
Name: params, dtype: float64

In [73]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:             log_return   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                      GARCH   Log-Likelihood:               -5133.77
Distribution:                  Normal   AIC:                           10275.5
Method:            Maximum Likelihood   BIC:                           10295.2
                                        No. Observations:                 1000
Date:                Sat, Apr 28 2018   Df Residuals:                      996
Time:                        00:45:39   Df Model:                            4
                               Mean Model                               
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu            -0.8465     20.863 -4.058e-02      0.968 [-41.738, 40.045]
                            Volatility Model                            
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          4.4414      5.509      0.806      0.420 [ -6.357, 15.240]
alpha[1]   6.3213e-09      0.208  3.037e-08      1.000 [ -0.408,  0.408]
beta[1]        1.0000  1.212e-02     82.488      0.000 [  0.976,  1.024]
========================================================================

Covariance estimator: robust
"""

In [74]:
fore_result = res11.forecast()
(fore_result.mean/1000)[-1:]

,h.1
1000,-0.000847


# One-Day Forward Volatility

In [75]:
forecasted_vol = 0.001 * np.sqrt(res.params['omega'] + res.params['alpha[1]'] * res.resid**2 + res.conditional_volatility**2 * res.params['beta[1]'])

In [76]:
model_fore = res.conditional_volatility*0.001

In [77]:
complete = np.c_[np.ones(len(forecasted_vol)),model_fore]

In [78]:
MZ_test = regression.linear_model.OLS(log_r**2,complete)

In [79]:
result = MZ_test.fit()

In [80]:
result.params

const   -0.012969
x1       0.380222
dtype: float64

In [81]:
dat = pd.read_csv("DEXJPUS.csv")

In [82]:
for i,v in enumerate(dat.DATE):
    if re.sub("-.*","",v) == "1990":
        break
    
dat = dat.loc[dat.DEXJPUS != "."].DEXJPUS.astype(float)
dat = np.array(dat)
dat = np.log(dat[1:])-np.log(dat[0:-1])
dat = dat[:,np.newaxis]
train = dat[:i]
test = dat[i:]

In [83]:
train.shape

(4326, 1)

In [84]:
garch11_train = arch_model(train*100, p=1,q=1)
res_train = garch11_train.fit(update_freq=5)

Iteration:      5,   Func. Count:     41,   Neg. LLF: 3493.691460700021
Iteration:     10,   Func. Count:     72,   Neg. LLF: 3479.226118808358
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3479.226126061366
            Iterations: 10
            Function evaluations: 72
            Gradient evaluations: 10


In [85]:
# Parameters of the training set
mu, omega, alpha, beta = res_train.params

In [86]:
#To extract residuals and the conditonal volatilities of the test set
garch11_test = arch_model(test*100, p=1,q=1)
res_test = garch11_test.fit(update_freq=5)

Iteration:      5,   Func. Count:     40,   Neg. LLF: 3074.762431321689
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3074.719556857562
            Iterations: 9
            Function evaluations: 67
            Gradient evaluations: 9


In [97]:
# Calculating the one-step forward prediction on the volatility.
pred = 0.01*np.sqrt(omega+alpha*res_test.resid**2+res_test.conditional_volatility**2*beta)
pred = pred.reshape(-1,1)

In [98]:
complete2 = np.c_[np.ones(len(pred)),pred]

In [99]:
MZ_test_DEXJPUS = regression.linear_model.OLS(test**2,complete2)

In [100]:
result2 = MZ_test_DEXJPUS.fit()

In [101]:
result2.params

array([-0.00024094,  0.04197132])